# simple LSTM network with keras

#### To run notebook install pandas(already comes with anaconda) and tensorflow(pip install tensorflow)

In [59]:
import numpy as np
import pandas as pd 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
import os 

In [60]:
#Read in the data 
path = 'appleData\\'
df1 = pd.read_csv(path+'aapl.txt')
df2 = pd.read_csv(path+'aapl2.txt')
df3 = pd.read_csv(path+'aapl3.txt')
#df1,df2,and df3 are from different days so we want to keep
#them separate when constructing the data set 

In [61]:
df1.head()

,bidPrice,bidSize,askPrice,askSize,lastSalePrice,lastSaleSize,volume,time
0,55.27,100,56.32,100,55.285,9,138776,8:18:14
1,55.00,100,56.32,100,55.360,100,139176,8:19:17
2,55.00,100,56.32,100,55.370,100,140787,8:21:55
3,55.00,100,56.32,100,55.365,500,143187,8:22:57
4,55.00,100,56.32,100,55.415,100,144188,8:23:58


In [77]:
#How many time steps we will use to try and predict the next price
#can be tuned freely. 
time_steps = 50
'''
NOTE: THERES A VERY A GOOD CHANCE I MESSED UP THIS FUNCTION SOMEWHERE. 

'''
def data_frames_to_train_data(df_lst,time_steps):
    
    #First will construct new features
    D = [np.zeros((len(df)-2,6)) for df in df_lst]
    for i,df in enumerate(df_lst):
        d = D[i]
        for j in range(1,len(df)-1):
            #last feature will be target
            features = [df.iloc[j,0]-df.iloc[j-1,0],df.iloc[j,1],df.iloc[j,2]-df.iloc[j-1,2],df.iloc[j,3]-df.iloc[j,3],
                          df.iloc[j,4]/df.iloc[j-1,4],df.iloc[j+1,4]/df.iloc[j,4]]
            
            d[j-1] = features
        #create train data
    X = np.zeros((sum([len(d) for d in D])-len(D)*(time_steps-1),time_steps,5))
    Y = np.zeros(sum([len(d) for d in D])-len(D)*(time_steps-1))
    count = 0
    for d in D:
        for i in range(len(d)-time_steps+1):
            X[count] = d[i:time_steps+i,:5]
            #print(Y.shape)
            Y[count] = d[i+time_steps-1,5]
            count+=1     
    return X,Y

    

In [88]:
#get train data
X,Y = data_frames_to_train_data([df1,df2,df3],time_steps)

In [96]:
#Standardizing Y helps a lot. Otherwise all the Y's are close to 1 and the network tries to map everything to 
#1.
mean_y = np.mean(Y)
std_y = np.std(Y)
Y = (Y-mean_y)/std_y

In [126]:
#build model
model = Sequential()
#model.add(LSTM(64,input_shape=(time_steps,5),return_sequences=True))
#model.add(Dropout(.2))
model.add(LSTM(64,input_shape=(time_steps,5)))
model.add(Dropout(.2))
model.add(Dense(1,activation='linear'))
model.compile(loss='mean_squared_error',optimizer='RMSprop')



In [127]:
model.fit(X,Y,epochs=15,validation_split=.1)

Train on 713 samples, validate on 80 samples
Epoch 1/15
713/713 [==============================] - 4s 5ms/sample - loss: 1.2148 - val_loss: 0.4730
Epoch 2/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.2438 - val_loss: 0.4468
Epoch 3/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1749 - val_loss: 0.3558
Epoch 4/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1789 - val_loss: 0.3548
Epoch 5/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1388 - val_loss: 0.3562
Epoch 6/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1321 - val_loss: 0.3502
Epoch 7/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1387 - val_loss: 0.3637
Epoch 8/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1658 - val_loss: 0.3508
Epoch 9/15
713/713 [==============================] - 1s 1ms/sample - loss: 1.1382 - val_loss: 0.3792
Epoch 10/15
713/713 [================

In [121]:
pred = model.predict(X)

In [128]:
pred*std_y + mean_y


array([[0.9999961 ],
       [0.9999959 ],
       [0.999996  ],
       [0.99999595],
       [0.99999225],
       [0.9999996 ],
       [0.9999909 ],
       [0.9999952 ],
       [1.0000012 ],
       [0.9999905 ],
       [0.9999979 ],
       [0.99999505],
       [0.9999996 ],
       [0.9999971 ],
       [0.99999535],
       [0.999996  ],
       [0.9999918 ],
       [1.        ],
       [0.9999966 ],
       [0.99999166],
       [0.9999966 ],
       [0.9999989 ],
       [0.99999297],
       [0.999999  ],
       [0.9999943 ],
       [0.99999756],
       [0.99999475],
       [0.999996  ],
       [0.999996  ],
       [0.999996  ],
       [0.9999952 ],
       [0.99999684],
       [1.0000243 ],
       [0.99999505],
       [0.99999726],
       [0.99999595],
       [0.9999944 ],
       [0.99999756],
       [0.99999654],
       [0.9999928 ],
       [0.9999984 ],
       [0.999996  ],
       [0.99999523],
       [0.99999684],
       [0.9999941 ],
       [0.9999981 ],
       [0.99999267],
       [0.999

In [123]:
Y*std_y + mean_y

array([1.00108333, 0.99765533, 1.        , 0.99755943, 1.        ,
       0.99882204, 0.99872993, 1.        , 1.00163503, 1.00063481,
       1.        , 0.99827805, 1.00081707, 0.9983672 , 1.00227149,
       1.00163177, 1.00117658, 0.999548  , 0.99927648, 1.        ,
       0.99963798, 1.0028067 , 0.99846515, 1.00027127, 1.00117519,
       1.0013544 , 1.00009017, 0.99855739, 1.00018059, 1.00054166,
       0.99972932, 1.00027076, 1.00090228, 0.9984675 , 0.99945829,
       1.00162602, 0.99882756, 0.99990971, 0.99954849, 1.00081308,
       1.00027081, 0.99963902, 1.        , 1.        , 0.99900695,
       0.99990963, 1.        , 0.99981925, 0.99819217, 1.00018111,
       1.00099593, 0.99945731, 1.00162896, 1.        , 1.00108421,
       0.99972924, 1.00027083, 1.        , 1.00018051, 1.00018047,
       1.00018044, 1.00027061, 0.99774551, 1.00081345, 1.00117403,
       1.00090204, 1.00126172, 1.        , 1.        , 1.00162016,
       1.00017973, 1.00017969, 1.00044916, 1.00143665, 1.00089

In [124]:
Y

array([ 1.67100924e+00, -3.62985306e+00, -4.18762636e-03, -3.77815041e+00,
       -4.18762636e-03, -1.82572641e+00, -1.96815826e+00, -4.18762636e-03,
        2.52412862e+00,  9.77441480e-01, -4.18762636e-03, -2.66691917e+00,
        1.25928189e+00, -2.52906367e+00,  3.50831989e+00,  2.51908620e+00,
        1.81520797e+00, -7.03132802e-01, -1.12300561e+00, -4.18762636e-03,
       -5.64001657e-01,  4.33594234e+00, -2.37759745e+00,  4.15293242e-01,
        1.81306983e+00,  2.09018660e+00,  1.35248470e-01, -2.23496401e+00,
        2.75062271e-01,  8.33410807e-01, -4.22760118e-01,  4.14498197e-01,
        1.39105401e+00, -2.37396021e+00, -8.41861682e-01,  2.51019661e+00,
       -1.81718384e+00, -1.43812575e-01, -7.02375412e-01,  1.25311807e+00,
        4.14573786e-01, -5.62385012e-01, -4.18762636e-03, -4.18762636e-03,
       -1.53978475e+00, -1.43926132e-01, -4.18762636e-03, -2.83689896e-01,
       -2.79971562e+00,  2.75871471e-01,  1.53585849e+00, -8.43377012e-01,
        2.51474753e+00, -

In [118]:
np.std(Y)

0.9999999999999998